In [27]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# mine custom stuff
from help_methods import flip_columns_and_rows

# for regressions with statsmodels:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.graphics.regressionplots import plot_leverage_resid2

# for regressions with scikit-learn:
import sklearn.linear_model as sklm
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report, precision_score, \
                            accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score

#for plotting confusion matrix:
import scikitplot as skplt

#for ordinal logistic regression
from mord import LogisticIT

#for KNN
# for KNN:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

In [4]:
#These are utility tools of the DMBA book. 
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score
from dmba import classificationSummary, gainsChart, liftChart

In [5]:
#for plotting decision trees
import pydotplus as pplus
import graphviz
from IPython.display import Image
from six import StringIO
# if not found: ! pip install six

#for modeling with decision trees
from sklearn.tree import export_graphviz

#Scikit stuff
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, \
                            BaggingClassifier, BaggingRegressor, \
                            GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, \
                            classification_report, precision_score, \
                            accuracy_score, roc_curve, roc_auc_score
import scikitplot as skplt


In [32]:
df = pd.read_excel("processed_CIR.xlsx")

df.reindex()

,Unnamed: 0,Segment,Category,Region,A,B,C,D,E,F,G,H,I,J,K,L,AVG,Year
0,2,INTERNET,Retail Price ($ per pair),NorthA,67.00,67.000,67.00,67.00,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,Y10
1,3,INTERNET,Search Engine Advert. ($000s),NorthA,5000.00,5000.000,5000.00,5000.00,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,Y10
2,4,INTERNET,Free Shipping,NorthA,0.00,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y10
3,6,INTERNET,S/Q Rating (1 to 10 stars),NorthA,4.00,4.000,4.00,4.00,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Y10
4,7,INTERNET,Model Availability,NorthA,200.00,200.000,200.00,200.00,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,Y10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,43,PRIVATELABEL,Offer Price (max = $40.00),LA,43.69,39.000,59.98,39.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.5,Y18
1076,44,PRIVATELABEL,S/Q Rating (min = 3.0 stars),LA,5.50,5.000,5.00,5.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Y18
1077,46,PRIVATELABEL,Pairs Offered (000s),LA,383.00,995.000,296.00,1003.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,Y18
1078,47,PRIVATELABEL,Pairs Sold (000s),LA,0.00,995.000,0.00,1003.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2677.0,Y18


In [33]:
master_pivot = flip_columns_and_rows(df)

In [34]:
master_pivot.columns = master_pivot.columns.str.replace(' ', '')

# Internet Segments     

In [35]:
idx = pd.IndexSlice
Internet_df = master_pivot.loc[idx[["Y11", "Y12", "Y13", "Y14", "Y15", "Y16", "Y17", "Y18"], :, "INTERNET"], :].dropna(axis=1)
Internet_df

Category              BrandAdvertising($000s)  \
Year Region Segment                             
Y11  AP     INTERNET                  10000.0   
     EP     INTERNET                  11000.0   
     LA     INTERNET                  10500.0   
     NorthA INTERNET                  12000.0   
     AP     INTERNET                  19500.0   
...                                       ...   
Y18  NorthA INTERNET                  30000.0   
     AP     INTERNET                  30000.0   
     EP     INTERNET                  30000.0   
     LA     INTERNET                  30000.0   
     NorthA INTERNET                  30000.0   

Category              BrandReputation(prior-yearimage)  CelebrityAppeal  \
Year Region Segment                                                       
Y11  AP     INTERNET                              70.0              0.0   
     EP     INTERNET                              70.0              0.0   
     LA     INTERNET                              70.0              0.0   
     NorthA INTERNET                              70.0              0.0   
     AP     INTERNET                              70.0              0.0   
...                                                ...              ...   
Y18  NorthA INTERNET                              95.0            230.0   
     AP     INTERNET                              79.0              0.0   
     EP     INTERNET                              79.0              0.0   
     LA     INTERNET                              79.0              0.0   
     NorthA INTERNET                              79.0              0.0   

Category              FreeShipping  MarketShare(%)  ModelAvailability  \
Year Region Segment                                                     
Y11  AP     INTERNET           0.0          0.2009              248.9   
     EP     INTERNET           0.0          0.1919              248.9   
     LA     INTERNET           0.0          0.1841              248.9   
     NorthA INTERNET           0.0          0.1789              248.8   
     AP     INTERNET           0.0          0.3722              347.6   
...                            ...             ...                ...   
Y18  NorthA INTERNET           1.0          0.3147              451.2   
     AP     INTERNET           1.0          0.2143              500.0   
     EP     INTERNET           1.0          0.2064              500.0   
     LA     INTERNET           1.0          0.2059              500.0   
     NorthA INTERNET           1.0          0.2269              500.0   

Category              OnlineOrders(000s)  PairsSold(000s)  \
Year Region Segment                                         
Y11  AP     INTERNET               217.0            217.0   
     EP     INTERNET               264.0            264.0   
     LA     INTERNET               201.0            201.0   
     NorthA INTERNET               287.0            287.0   
     AP     INTERNET               402.0            402.0   
...                                  ...              ...   
Y18  NorthA INTERNET              1022.0           1022.0   
     AP     INTERNET               631.0            631.0   
     EP     INTERNET               597.0            597.0   
     LA     INTERNET               601.0            601.0   
     NorthA INTERNET               737.0            737.0   

Category              RetailPrice($perpair)  S/QRating(1to10stars)  \
Year Region Segment                                                  
Y11  AP     INTERNET                   75.0                    5.5   
     EP     INTERNET                   77.0                    5.5   
     LA     INTERNET                   77.0                    5.5   
     NorthA INTERNET                   75.0                    5.5   
     AP     INTERNET                   83.0                    6.7   
...                                     ...                    ...   
Y18  NorthA INTERNET                  117.5                    9.9   
     AP     

In [36]:
print(Internet_df.dtypes)

Category
BrandAdvertising($000s)             float64
BrandReputation(prior-yearimage)    float64
CelebrityAppeal                     float64
FreeShipping                        float64
MarketShare(%)                      float64
ModelAvailability                   float64
OnlineOrders(000s)                  float64
PairsSold(000s)                     float64
RetailPrice($perpair)               float64
S/QRating(1to10stars)               float64
SearchEngineAdvert.($000s)          float64
dtype: object


In [37]:
y = Internet_df["OnlineOrders(000s)"]
X_values = Internet_df.drop(columns=["OnlineOrders(000s)", "BrandReputation(prior-yearimage)", "MarketShare(%)", "PairsSold(000s)"])
#X_values = sm.add_constant(X_values)
X_train, X_test, Y_train, Y_test = train_test_split(X_values, y, test_size=0.25, random_state=7)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(96, 7) (96,)
(32, 7) (32,)


In [38]:
X_train

,,Category,BrandAdvertising($000s),CelebrityAppeal,FreeShipping,ModelAvailability,RetailPrice($perpair),S/QRating(1to10stars),SearchEngineAdvert.($000s)
Year,Region,Segment,,,,,,,
Y16,NorthA,INTERNET,30000.0,245.0,0.0,499.9,111.75,8.0,20000.0
Y11,NorthA,INTERNET,16500.0,0.0,1.0,297.3,96.00,5.9,7250.0
Y12,AP,INTERNET,26500.0,100.0,0.0,445.3,97.00,7.1,11250.0
Y18,EP,INTERNET,30000.0,85.0,0.0,500.0,100.00,8.5,20000.0
Y13,NorthA,INTERNET,30000.0,215.0,1.0,444.2,113.99,8.6,18500.0
...,...,...,...,...,...,...,...,...,...
Y15,NorthA,INTERNET,8000.0,0.0,1.0,256.7,102.50,9.1,10250.0
Y18,LA,INTERNET,30000.0,100.0,0.0,500.0,95.50,8.4,20000.0
Y12,EP,INTERNET,22500.0,215.0,0.0,390.5,114.90,8.4,11000.0


In [39]:
OLS_internet_all = sm.OLS(y, X_values)
OLS_internet_all = OLS_internet_all.fit()
OLS_internet_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     OnlineOrders(000s)   R-squared (uncentered):                   0.962
Model:                            OLS   Adj. R-squared (uncentered):              0.960
Method:                 Least Squares   F-statistic:                              439.6
Date:                Mon, 30 Jan 2023   Prob (F-statistic):                    7.67e-83
Time:                        19:43:46   Log-Likelihood:                         -782.90
No. Observations:                 128   AIC:                                      1580.
Df Residuals:                     121   BIC:                                      1600.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
BrandAdvertising($000s)        0.0052      0.002      2.503      0.014       0.001       0.009
CelebrityAppeal                0.8098      0.135      5.996      0.000       0.542       1.077
FreeShipping                  23.7067     26.361      0.899      0.370     -28.481      75.895
ModelAvailability              0.6137      0.092      6.706      0.000       0.433       0.795
RetailPrice($perpair)         -6.6714      1.197     -5.573      0.000      -9.041      -4.302
S/QRating(1to10stars)         64.2773     17.084      3.762      0.000      30.455      98.100
SearchEngineAdvert.($000s)     0.0185      0.003      5.827      0.000       0.012       0.025
==============================================================================
Omnibus:                        0.041   Durbin-Watson:                   1.050
Prob(Omnibus):                  0.980   Jarque-Bera (JB):                0.116
Skew:                           0.041   Prob(JB):                        0.944
Kurtosis:                       2.878   Cond. No.                     8.08e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 8.08e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
preds = OLS_internet_all.predict(X_train)
regressionSummary(Y_train, preds)


Regression statistics

                      Mean Error (ME) : 0.3197
       Root Mean Squared Error (RMSE) : 107.7555
            Mean Absolute Error (MAE) : 88.3418
          Mean Percentage Error (MPE) : -1.3708
Mean Absolute Percentage Error (MAPE) : 20.0251


In [41]:
preds = OLS_internet_all.predict(X_test)
regressionSummary(Y_test, preds)


Regression statistics

                      Mean Error (ME) : 16.9167
       Root Mean Squared Error (RMSE) : 115.2116
            Mean Absolute Error (MAE) : 90.9474
          Mean Percentage Error (MPE) : 0.0303
Mean Absolute Percentage Error (MAPE) : 18.3000


Can we do better with Ensembles? 

# Ensembles 



In [42]:
y_values = Internet_df["OnlineOrders(000s)"]
X_values = Internet_df.drop(columns=["OnlineOrders(000s)", "BrandReputation(prior-yearimage)", "MarketShare(%)", "PairsSold(000s)"])
#X_values = sm.add_constant(X_values)
X_train, X_test, Y_train, Y_test = train_test_split(X_values, y, test_size=0.25, random_state=7)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(96, 7) (96,)
(32, 7) (32,)


In [43]:
bag = RandomForestRegressor(max_features=11, random_state=23)

In [44]:
bag.fit(X_train, Y_train)

RandomForestRegressor(max_features=11, random_state=23)

In [45]:
bag_preds = bag.predict(X_train)
bag_preds

array([ 934.4 ,  344.56,  474.77,  935.35,  655.3 ,  454.94,  243.6 ,
        200.44,  251.68,  470.77,  333.54,  490.36,  226.11,  240.73,
        897.72,  213.61,  480.3 ,  345.55,  366.72,  341.36,  376.75,
        325.99,  622.41,  400.68,  756.21,  524.68,  553.64,  463.57,
        382.93,  619.1 ,  262.25,  260.77,  737.9 ,  428.52,  253.61,
        516.7 ,  650.93,  599.78,  282.39,  215.23,  932.37,  717.24,
        551.21,  424.43,  292.56,  547.68,  238.71,  474.66,  559.59,
        901.35,  290.61,  392.06,  388.05,  493.21,  507.49,  453.28,
        257.18,  232.24,  597.58,  694.44, 1015.77,  324.58,  547.2 ,
        796.88,  737.55,  587.21,  450.  ,  375.97,  335.72,  675.22,
        678.82,  231.21,  455.72,  504.35,  226.48,  637.13, 1028.11,
        437.75,  235.66,  308.2 ,  431.38,  521.74,  330.93,  518.82,
        953.24,  435.29,  519.15,  468.69, 1003.41,  562.97,  436.98,
        260.24,  976.38,  397.08,  775.23,  708.01])

In [46]:
regressionSummary(Y_train, bag_preds)


Regression statistics

                      Mean Error (ME) : -2.5819
       Root Mean Squared Error (RMSE) : 36.0280
            Mean Absolute Error (MAE) : 27.7840
          Mean Percentage Error (MPE) : -2.1046
Mean Absolute Percentage Error (MAPE) : 6.3747


In [47]:
NRMSE = 36.0280 / (float(max(Y_train)) - float(min(Y_train))) 
NRMSE

0.04094090909090909

In [48]:
bag_preds = bag.predict(X_test)
regressionSummary(Y_test, bag_preds)


Regression statistics

                      Mean Error (ME) : -2.4166
       Root Mean Squared Error (RMSE) : 84.5681
            Mean Absolute Error (MAE) : 66.2491
          Mean Percentage Error (MPE) : -3.8197
Mean Absolute Percentage Error (MAPE) : 12.5695


In [49]:
NRMSE = 84.5681 / (float(max(Y_train)) - float(min(Y_train))) 
NRMSE

0.09610011363636364

The ensemble had a .09 Normalized RMSE 

# Decision Tree 

In [50]:
tree_reg = DecisionTreeRegressor(max_depth = 5, random_state=23)

In [51]:
tree_reg.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=5, random_state=23)

In [52]:
tree_preds = tree_reg.predict(X_train)

In [53]:
regressionSummary(Y_train, tree_preds)


Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 48.9314
            Mean Absolute Error (MAE) : 34.7656
          Mean Percentage Error (MPE) : -1.1432
Mean Absolute Percentage Error (MAPE) : 7.9815


we will stick with ensembles sine it had a lower RMSE on the test data.

# Test Ensemble with practice round data for Year 11 and 12

In [82]:
df = pd.read_excel("processed_practice_CIR.xlsx")
df.reindex()
pivot = flip_columns_and_rows(df)
pivot.columns = pivot.columns.str.replace(' ', '')
pivot

Category                  Gained/Lost(duetostockouts)  \
Year Region Segment                                     
Y11  AP     INTERNET                              NaN   
            PRIVATELABEL                          NaN   
            WHOLESALE                          -164.0   
     EP     INTERNET                              NaN   
            PRIVATELABEL                          NaN   
...                                               ...   
Y12  LA     PRIVATELABEL                          NaN   
            WHOLESALE                            -1.0   
     NorthA INTERNET                              NaN   
            PRIVATELABEL                          NaN   
            WHOLESALE                           345.0   

Category                  BrandAdvertising($000s)  \
Year Region Segment                                 
Y11  AP     INTERNET                       6000.0   
            PRIVATELABEL                      NaN   
            WHOLESALE                      6000.0   
     EP     INTERNET                       6000.0   
            PRIVATELABEL                      NaN   
...                                           ...   
Y12  LA     PRIVATELABEL                      NaN   
            WHOLESALE                     28000.0   
     NorthA INTERNET                      10000.0   
            PRIVATELABEL                      NaN   
            WHOLESALE                     10000.0   

Category                  BrandReputation(prior-yearimage)  CelebrityAppeal  \
Year Region Segment                                                           
Y11  AP     INTERNET                                  70.0              0.0   
            PRIVATELABEL                               NaN              NaN   
            WHOLESALE                                 70.0              0.0   
     EP     INTERNET                                  70.0              0.0   
            PRIVATELABEL                               NaN              NaN   
...                                                    ...              ...   
Y12  LA     PRIVATELABEL                               NaN              NaN   
            WHOLESALE                                 60.0              0.0   
     NorthA INTERNET                                  60.0              0.0   
            PRIVATELABEL                               NaN              NaN   
            WHOLESALE                                 60.0              0.0   

Category                  DeliveryTime(weeks)  FreeShipping  MarketShare(%)  \
Year Region Segment                                                           
Y11  AP     INTERNET                      NaN           1.0          0.3898   
            PRIVATELABEL                  NaN           NaN          0.3190   
            WHOLESALE                     3.0           NaN          0.2885   
     EP     INTERNET                      NaN           1.0          0.3616   
            PRIVATELABEL                  NaN           NaN          0.2160   
...                                       ...           ...             ...   
Y12  LA     PRIVATELABEL                  NaN           NaN          0.3000   
            WHOLESALE                     2.0           NaN          0.2589   
     NorthA INTERNET                      NaN           0.0          0.1965   
            PRIVATELABEL                  NaN           NaN          0.2470   
            WHOLESALE                     2.0           NaN          0.2082   

Category                  ModelAvailability  OfferPrice(max=$40.00)  \
Year Region Segment                                                   
Y11  AP     INTERNET                  248.9                     NaN   
            PRIVATELABEL                NaN                    32.0   
            WHOLESALE                 248.9                     NaN   
     EP     INTERNET                  248.8                     NaN   
            PRIVATELABEL                NaN                    37.0   
...            

In [83]:
idx = pd.IndexSlice
Internet_practice_df = pivot.loc[idx[["Y11", "Y12"], :, "INTERNET"], :].dropna(axis=1)
Internet_practice_df

Category              BrandAdvertising($000s)  \
Year Region Segment                             
Y11  AP     INTERNET                   6000.0   
     EP     INTERNET                   6000.0   
     LA     INTERNET                   6000.0   
     NorthA INTERNET                   7500.0   
     AP     INTERNET                  16000.0   
     EP     INTERNET                  19500.0   
     LA     INTERNET                  16000.0   
     NorthA INTERNET                  24000.0   
     AP     INTERNET                   8500.0   
     EP     INTERNET                   9500.0   
     LA     INTERNET                   8000.0   
     NorthA INTERNET                  10500.0   
     AP     INTERNET                  18000.0   
     EP     INTERNET                  18000.0   
     LA     INTERNET                  18000.0   
     NorthA INTERNET                  18000.0   
Y12  AP     INTERNET                   6500.0   
     EP     INTERNET                   6500.0   
     LA     INTERNET                   6500.0   
     NorthA INTERNET                   8000.0   
     AP     INTERNET                  10000.0   
     EP     INTERNET                  10000.0   
     LA     INTERNET                  11000.0   
     NorthA INTERNET                  10000.0   
     AP     INTERNET                  14000.0   
     EP     INTERNET                  30000.0   
     LA     INTERNET                  14500.0   
     NorthA INTERNET                  30000.0   
     AP     INTERNET                  28000.0   
     EP     INTERNET                  10000.0   
     LA     INTERNET                  28000.0   
     NorthA INTERNET                  10000.0   

Category              BrandReputation(prior-yearimage)  CelebrityAppeal  \
Year Region Segment                                                       
Y11  AP     INTERNET                              70.0              0.0   
     EP     INTERNET                              70.0              0.0   
     LA     INTERNET                              70.0              0.0   
     NorthA INTERNET                              70.0              0.0   
     AP     INTERNET                              70.0              0.0   
     EP     INTERNET                              70.0              0.0   
     LA     INTERNET                              70.0              0.0   
     NorthA INTERNET                              70.0              0.0   
     AP     INTERNET                              70.0              0.0   
     EP     INTERNET                              70.0              0.0   
     LA     INTERNET                              70.0              0.0   
     NorthA INTERNET                              70.0              0.0   
     AP     INTERNET                              70.0              0.0   
     EP     INTERNET                              70.0              0.0   
     LA     INTERNET                              70.0              0.0   
     NorthA INTERNET                              70.0              0.0   
Y12  AP     INTERNET                              82.0             60.0   
     EP     INTERNET                              82.0             50.0   
     LA     INTERNET                              82.0             95.0   
     NorthA INTERNET                              82.0             60.0   
     AP     INTERNET                              73.0             60.0   
     EP     INTERNET                              73.0             85.0   
     LA     INTERNET                              73.0            100.0   
     NorthA INTERNET                              73.0             40.0   
     AP     INTERNET                              65.0            165.0   
     EP     INTERNET                              65.0            115.0   
     LA     INTERNET                              65.0            140.0   
     NorthA INTERNET                              65.0            160.0   
     AP     INTERNET                              60.0              0.0   
     EP 

In [87]:
y_values = Internet_practice_df["OnlineOrders(000s)"]
X_values = Internet_practice_df.drop(columns=["OnlineOrders(000s)", "BrandReputation(prior-yearimage)", "MarketShare(%)", "PairsSold(000s)"])
y_values

Year  Region  Segment 
Y11   AP      INTERNET    396.0
      EP      INTERNET    474.0
      LA      INTERNET    365.0
      NorthA  INTERNET    595.0
      AP      INTERNET    152.0
      EP      INTERNET    247.0
      LA      INTERNET    180.0
      NorthA  INTERNET    284.0
      AP      INTERNET    244.0
      EP      INTERNET    301.0
      LA      INTERNET    244.0
      NorthA  INTERNET    357.0
      AP      INTERNET    224.0
      EP      INTERNET    289.0
      LA      INTERNET    227.0
      NorthA  INTERNET    316.0
Y12   AP      INTERNET    428.0
      EP      INTERNET    537.0
      LA      INTERNET    464.0
      NorthA  INTERNET    722.0
      AP      INTERNET     59.0
      EP      INTERNET    112.0
      LA      INTERNET     86.0
      NorthA  INTERNET     96.0
      AP      INTERNET    333.0
      EP      INTERNET    492.0
      LA      INTERNET    268.0
      NorthA  INTERNET    548.0
      AP      INTERNET    348.0
      EP      INTERNET    311.0
      LA      INT

In [85]:
bag.predict(X_values)
regressionSummary(y_values, bag_preds)



Regression statistics

                      Mean Error (ME) : -236.1353
       Root Mean Squared Error (RMSE) : 330.5387
            Mean Absolute Error (MAE) : 253.9722
          Mean Percentage Error (MPE) : -124.7095
Mean Absolute Percentage Error (MAPE) : 128.9279


In [86]:
NRMSE = 330.5387 / (float(max(y_values)) - float(min(y_values))) 
NRMSE

0.4985500754147813

Using the actual practice data increase the normalized RMSE to .5 which is getting to the point of not being great 